In [1]:
bucket_name = 'sessac-project-13-bucket'
mount_path = '/home/jupyter/dataset'
!gcsfuse --implicit-dirs {bucket_name} {mount_path}

{"time":"22/11/2023 08:48:58.582196","severity":"INFO","msg":"Start gcsfuse/1.2.0 (Go version go1.21.0) for app \"\" using mount point: /home/jupyter/dataset\n"}


In [2]:
from glob import glob
import os
import numpy as np
import pandas as pd

In [3]:
HOME = os.getcwd()
basePath = '{}/dataset/highway_total/train/label'.format(HOME)
# tobasePath = '{}/highway/bbox_highway/train/labels'.format(HOME)

train_filename = []
for filepath in glob(os.path.join(basePath, '*.xml')):
    filename = os.path.basename(filepath)
    filename = filename[:-4]
    train_filename.append(filename)

len(train_filename)

405

In [4]:
def glob_files(path, file_type="*"):
    search_string = os.path.join(path, file_type)
    files = glob(search_string)

    # print('searching ', path)
    paths = []
    for f in files:
      if os.path.isdir(f):
        sub_paths = glob_files(f + '/')
        paths += sub_paths
      else:
        paths.append(f)

    # We sort the images in alphabetical order to match them
    #  to the annotation files
    paths.sort()

    return paths


WIDTH = 1
HEIGHT = 1

def load_labels(path):
  files = glob_files(path, "*.txt")

  Y_data = []
  for file in files:
    with open(file) as f:
      lines = f.readlines()

      boxes = []
      for line in lines:
        tokens = line.split()

        class_id = int(tokens[0])
        xc = float(tokens[1]) * WIDTH
        yc = float(tokens[2]) * HEIGHT
        width = float(tokens[3]) * WIDTH
        height = float(tokens[4]) * HEIGHT

        boxes.append(np.array([class_id, xc, yc, width, height]))
        # print(class_id, xc, yc, width, height)

      Y_data.append(np.array(boxes))
      # print(lines)
  return Y_data  ## 이미지마다 탐지 대상 개수가 달라서(행 개수가 달라서) 정해진 shape의 넘파이 어레이로 변환할 수 없다! 변환하려면 패딩 등이 필요.

In [6]:
classes = ['car', 'bus', 'truck']
valid_total_df = pd.DataFrame()

for foldername in train_filename:
    Y_test = load_labels(os.path.join(basePath, foldername))
    folderPath = os.path.join(basePath, foldername)
    txtfilenameList = glob(os.path.join(folderPath, '*.txt'))
    txt_df = pd.DataFrame(txtfilenameList)
    txt_df.columns = ['txt_in_train_folder']
    for i in range(len(txt_df)):
        df_ex = pd.DataFrame(pd.DataFrame(Y_test[i])[0].value_counts()).T
        for cls in df_ex.columns:
            # print(float(i))
            col = classes[int(cls)] + '_cnt'
            # print(col)
            txt_df.loc[i, col] = df_ex.loc['count', cls]
    # txt_df.dropna(inplace=True)
    valid_total_df = pd.concat([valid_total_df, txt_df])

valid_total_df.reset_index(drop=True, inplace=True)
valid_total_df

,txt_in_train_folder,truck_cnt,car_cnt,bus_cnt
0,/home/jupyter/dataset/highway_total/train/labe...,14.0,14.0,NaN
1,/home/jupyter/dataset/highway_total/train/labe...,16.0,14.0,NaN
2,/home/jupyter/dataset/highway_total/train/labe...,21.0,23.0,NaN
3,/home/jupyter/dataset/highway_total/train/labe...,17.0,12.0,2.0
4,/home/jupyter/dataset/highway_total/train/labe...,18.0,17.0,2.0
...,...,...,...,...
56128,/home/jupyter/dataset/highway_total/train/labe...,NaN,2.0,NaN
56129,/home/jupyter/dataset/highway_total/train/labe...,NaN,2.0,NaN
56130,/home/jupyter/dataset/highway_total/train/labe...,NaN,3.0,NaN
56131,/home/jupyter/dataset/highway_total/train/labe...,NaN,8.0,NaN


In [7]:
valid_total_df.to_csv('train_total_txt_label.csv', index=False)

In [8]:
condition1 = valid_total_df['car_cnt'].isnull()
condition2 = valid_total_df['truck_cnt'].isnull()
valid_total_df[(condition1) & (condition2)]

,txt_in_train_folder,truck_cnt,car_cnt,bus_cnt
5761,/home/jupyter/dataset/highway_total/train/labe...,NaN,NaN,1.0
5804,/home/jupyter/dataset/highway_total/train/labe...,NaN,NaN,1.0
5810,/home/jupyter/dataset/highway_total/train/labe...,NaN,NaN,1.0
5841,/home/jupyter/dataset/highway_total/train/labe...,NaN,NaN,1.0
5951,/home/jupyter/dataset/highway_total/train/labe...,NaN,NaN,1.0
...,...,...,...,...
50832,/home/jupyter/dataset/highway_total/train/labe...,NaN,NaN,1.0
50838,/home/jupyter/dataset/highway_total/train/labe...,NaN,NaN,1.0
50896,/home/jupyter/dataset/highway_total/train/labe...,NaN,NaN,1.0
50968,/home/jupyter/dataset/highway_total/train/labe...,NaN,NaN,1.0


In [9]:
busonlyList = list(valid_total_df[(condition1) & (condition2)]['txt_in_train_folder'].values)

for i in busonlyList:
    basename = os.path.basename(i)
    print(basename)

Suwon_CH01_20201213_0700_SUN_9m_NH_highway_TW5_snow_FHD_015.txt
Suwon_CH01_20201213_0700_SUN_9m_NH_highway_TW5_snow_FHD_059.txt
Suwon_CH01_20201213_0700_SUN_9m_NH_highway_TW5_snow_FHD_065.txt
Suwon_CH01_20201213_0700_SUN_9m_NH_highway_TW5_snow_FHD_097.txt
Suwon_CH01_20201213_0730_SUN_9m_NH_highway_TW5_snow_FHD_003.txt
Suwon_CH01_20201213_0730_SUN_9m_NH_highway_TW5_snow_FHD_087.txt
Suwon_CH02_20201213_0732_SUN_9m_NH_highway_TW5_snow_FHD_001.txt
Suwon_CH02_20201213_0732_SUN_9m_NH_highway_TW5_snow_FHD_012.txt
Suwon_CH02_20201213_0732_SUN_9m_NH_highway_TW5_snow_FHD_013.txt
Suwon_CH02_20201213_0732_SUN_9m_NH_highway_TW5_snow_FHD_171.txt
Suwon_CH03_20200720_2000_MON_9m_NH_highway_OW5_sunny_FHD_073.txt
Suwon_CH03_20201213_0704_SUN_9m_NH_highway_OW5_snow_FHD_054.txt
Suwon_CH03_20201213_0704_SUN_9m_NH_highway_OW5_snow_FHD_055.txt
Suwon_CH03_20201213_0704_SUN_9m_NH_highway_OW5_snow_FHD_061.txt
Suwon_CH03_20201213_0704_SUN_9m_NH_highway_OW5_snow_FHD_120.txt
Suwon_CH03_20201213_0704_SUN_9m_NH_high